In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("fist dataframe").getOrCreate()

dataframe_1 = spark.createDataFrame([("Pedro", 10), ("Maria", 20), ("José", 40)])
dataframe_1.show()

24/04/01 09:46:13 WARN Utils: Your hostname, IdeaPad-Gaming-3-15IHU6 resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
24/04/01 09:46:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/01 09:46:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+---+
|   _1| _2|
+-----+---+
|Pedro| 10|
|Maria| 20|
| José| 40|
+-----+---+



In [2]:
schema = "Id INT, Name STRING"
data = [[1, "Pedro"], [2, "Maria"]]
dataframe_2 = spark.createDataFrame(data, schema)
dataframe_2.show()

+---+-----+
| Id| Name|
+---+-----+
|  1|Pedro|
|  2|Maria|
+---+-----+



In [3]:
from pyspark.sql.functions import sum

schema_2 = "Produtos String, Vendas INT"
vendas = [["Pencil", 10], ["Pen", 20], ["Pencil", 15]]
dataframe_3 = spark.createDataFrame(vendas, schema_2)
dataframe_3.show()

agg = dataframe_3.groupBy("Produtos").agg(sum("Vendas"))
agg.show()

+--------+------+
|Produtos|Vendas|
+--------+------+
|  Pencil|    10|
|     Pen|    20|
|  Pencil|    15|
+--------+------+

+--------+-----------+
|Produtos|sum(Vendas)|
+--------+-----------+
|     Pen|         20|
|  Pencil|         25|
+--------+-----------+



In [4]:
dataframe_3.select("Produtos").show()

+--------+
|Produtos|
+--------+
|  Pencil|
|     Pen|
|  Pencil|
+--------+



In [5]:
from pyspark.sql.functions import expr

dataframe_3.select("Produtos", "Vendas", expr("Vendas * 0.2")).show()

+--------+------+--------------+
|Produtos|Vendas|(Vendas * 0.2)|
+--------+------+--------------+
|  Pencil|    10|           2.0|
|     Pen|    20|           4.0|
|  Pencil|    15|           3.0|
+--------+------+--------------+



In [7]:
print(dataframe_3.schema)
dataframe_3.columns

StructType([StructField('Produtos', StringType(), True), StructField('Vendas', IntegerType(), True)])


['Produtos', 'Vendas']

In [17]:
from pathlib import Path
from pyspark.sql.types import *

path = Path().home() / "Documents/PySparkCurso/download/despachantes.csv"

arq_schema = (
    "id STRING, name STRING, status STRING, city STRING, " "sales INT, date STRING"
)

despachantes = spark.read.csv(str(path), schema=arq_schema)
despachantes.show()
print(despachantes.schema)

+---+-------------------+------+-------------+-----+----------+
| id|               name|status|         city|sales|      date|
+---+-------------------+------+-------------+-----+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|   23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|   34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|   34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|   36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|   12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|   22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|   45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|   65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|   54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|    0|2020-09-05|
+---+-------------------+------+-------------+-----+----------+

StructType([StructField('id', StringType(), True), StructField('name', StringType(), True), StructField

In [16]:
desp_autoshcema = spark.read.load(
    str(path), header=False, format="csv", sep=",", inferSchema=True
)

desp_autoshcema.show()
print(desp_autoshcema.schema)

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+

StructType([StructField('_c0', IntegerType(), True), StructField('_c1', StringType(), True), StructField('_c2', StringType(), True), StructField(

In [19]:
from pyspark.sql import functions as func

despachantes.select("id", "name", "sales").where(func.col("sales") > 20).show()

+---+-------------------+-----+
| id|               name|sales|
+---+-------------------+-----+
|  1|   Carminda Pestana|   23|
|  2|    Deolinda Vilela|   34|
|  3|   Emídio Dornelles|   34|
|  4|Felisbela Dornelles|   36|
|  6|   Matilde Rebouças|   22|
|  7|    Noêmia   Orriça|   45|
|  8|      Roque Vásquez|   65|
|  9|      Uriel Queiroz|   54|
+---+-------------------+-----+



In [20]:
despachantes.select("id", "name", "sales").where(
    (func.col("sales") > 20) & (func.col("sales") < 40)
).show()

+---+-------------------+-----+
| id|               name|sales|
+---+-------------------+-----+
|  1|   Carminda Pestana|   23|
|  2|    Deolinda Vilela|   34|
|  3|   Emídio Dornelles|   34|
|  4|Felisbela Dornelles|   36|
|  6|   Matilde Rebouças|   22|
+---+-------------------+-----+



In [21]:
new_df = despachantes.withColumnRenamed("name", "names")
new_df.columns

['id', 'names', 'status', 'city', 'sales', 'date']

In [22]:
despachantes_2 = despachantes.withColumn(
    "data_2", func.to_timestamp(func.col("date"), "yyyy-MM-dd")
)

despachantes_2.schema

StructType([StructField('id', StringType(), True), StructField('name', StringType(), True), StructField('status', StringType(), True), StructField('city', StringType(), True), StructField('sales', IntegerType(), True), StructField('date', StringType(), True), StructField('data_2', TimestampType(), True)])

In [26]:
despachantes_2.select(func.year("data_2")).distinct().show()

+------------+
|year(data_2)|
+------------+
|        2018|
|        2019|
|        2020|
+------------+



In [27]:
despachantes_2.select("name", func.year("data_2")).orderBy("name").show()

+-------------------+------------+
|               name|year(data_2)|
+-------------------+------------+
|   Carminda Pestana|        2020|
|    Deolinda Vilela|        2020|
|   Emídio Dornelles|        2020|
|Felisbela Dornelles|        2020|
|     Graça Ornellas|        2020|
|   Matilde Rebouças|        2019|
|    Noêmia   Orriça|        2019|
|      Roque Vásquez|        2020|
|      Uriel Queiroz|        2018|
|   Viviana Sequeira|        2020|
+-------------------+------------+



In [28]:
despachantes_2.select("data_2").groupBy(func.year("data_2")).count().show()

+------------+-----+
|year(data_2)|count|
+------------+-----+
|        2018|    1|
|        2019|    2|
|        2020|    7|
+------------+-----+



In [29]:
despachantes_2.select(func.sum("sales")).show()

+----------+
|sum(sales)|
+----------+
|       325|
+----------+



In [30]:
despachantes.take(1)

[Row(id='1', name='Carminda Pestana', status='Ativo', city='Santa Maria', sales=23, date='2020-08-11')]

In [31]:
despachantes.collect()

[Row(id='1', name='Carminda Pestana', status='Ativo', city='Santa Maria', sales=23, date='2020-08-11'),
 Row(id='2', name='Deolinda Vilela', status='Ativo', city='Novo Hamburgo', sales=34, date='2020-03-05'),
 Row(id='3', name='Emídio Dornelles', status='Ativo', city='Porto Alegre', sales=34, date='2020-02-05'),
 Row(id='4', name='Felisbela Dornelles', status='Ativo', city='Porto Alegre', sales=36, date='2020-02-05'),
 Row(id='5', name='Graça Ornellas', status='Ativo', city='Porto Alegre', sales=12, date='2020-02-05'),
 Row(id='6', name='Matilde Rebouças', status='Ativo', city='Porto Alegre', sales=22, date='2019-01-05'),
 Row(id='7', name='Noêmia   Orriça', status='Ativo', city='Santa Maria', sales=45, date='2019-10-05'),
 Row(id='8', name='Roque Vásquez', status='Ativo', city='Porto Alegre', sales=65, date='2020-03-05'),
 Row(id='9', name='Uriel Queiroz', status='Ativo', city='Porto Alegre', sales=54, date='2018-05-05'),
 Row(id='10', name='Viviana Sequeira', status='Ativo', city='Po

In [32]:
despachantes.count()

10

In [38]:
len(despachantes.columns)

6

In [39]:
despachantes.show()

+---+-------------------+------+-------------+-----+----------+
| id|               name|status|         city|sales|      date|
+---+-------------------+------+-------------+-----+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|   23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|   34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|   34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|   36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|   12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|   22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|   45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|   65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|   54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|    0|2020-09-05|
+---+-------------------+------+-------------+-----+----------+



In [42]:
despachantes.orderBy(func.col("sales").desc()).show()

+---+-------------------+------+-------------+-----+----------+
| id|               name|status|         city|sales|      date|
+---+-------------------+------+-------------+-----+----------+
|  8|      Roque Vásquez| Ativo| Porto Alegre|   65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|   54|2018-05-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|   45|2019-10-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|   36|2020-02-05|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|   34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|   34|2020-02-05|
|  1|   Carminda Pestana| Ativo|  Santa Maria|   23|2020-08-11|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|   22|2019-01-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|   12|2020-02-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|    0|2020-09-05|
+---+-------------------+------+-------------+-----+----------+



In [43]:
despachantes.orderBy(func.col("city").desc(), func.col("sales").desc()).show()

+---+-------------------+------+-------------+-----+----------+
| id|               name|status|         city|sales|      date|
+---+-------------------+------+-------------+-----+----------+
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|   45|2019-10-05|
|  1|   Carminda Pestana| Ativo|  Santa Maria|   23|2020-08-11|
|  8|      Roque Vásquez| Ativo| Porto Alegre|   65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|   54|2018-05-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|   36|2020-02-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|   34|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|   22|2019-01-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|   12|2020-02-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|    0|2020-09-05|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|   34|2020-03-05|
+---+-------------------+------+-------------+-----+----------+



In [46]:
despachantes.groupBy("city").agg(func.sum("sales")).show()

+-------------+----------+
|         city|sum(sales)|
+-------------+----------+
|  Santa Maria|        68|
|Novo Hamburgo|        34|
| Porto Alegre|       223|
+-------------+----------+



In [48]:
despachantes.groupBy("city").agg(func.sum("sales").alias("total_sales")).orderBy(
    func.col("total_sales").desc()
).show()

+-------------+-----------+
|         city|total_sales|
+-------------+-----------+
| Porto Alegre|        223|
|  Santa Maria|         68|
|Novo Hamburgo|         34|
+-------------+-----------+



In [50]:
despachantes.filter(func.col("city") == "Porto Alegre").where(
    func.col("sales") > 20
).show()

+---+-------------------+------+------------+-----+----------+
| id|               name|status|        city|sales|      date|
+---+-------------------+------+------------+-----+----------+
|  3|   Emídio Dornelles| Ativo|Porto Alegre|   34|2020-02-05|
|  4|Felisbela Dornelles| Ativo|Porto Alegre|   36|2020-02-05|
|  6|   Matilde Rebouças| Ativo|Porto Alegre|   22|2019-01-05|
|  8|      Roque Vásquez| Ativo|Porto Alegre|   65|2020-03-05|
|  9|      Uriel Queiroz| Ativo|Porto Alegre|   54|2018-05-05|
+---+-------------------+------+------------+-----+----------+

